<a href="https://colab.research.google.com/github/florescl/composer/blob/laura%2Ftpu-perf/TPU_Training_in_composer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

::## TPU Training in composer

### [RUNME] Install Colab compatible PyTorch/XLA wheels and dependencies



In [ ]:
!pip install cloud-tpu-client==0.10 torch==1.12.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.12-cp37-cp37m-linux_x86_64.whl

In [1]:
# Start training processes


%pip install git+https://github.com/florescl/composer.git@laura/tpu-perf

from composer import Trainer
from composer import models
from composer.models import ComposerClassifier
from torchvision.models import resnet
import torch
model = ComposerClassifier(module=resnet.resnet18())

from torchvision import datasets, transforms

data_directory = "../data"

# Normalization constants
mean = (0.507, 0.487, 0.441)
std = (0.267, 0.256, 0.276)

batch_size = 1024

cifar10_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])

train_dataset = datasets.CIFAR10(data_directory, train=True, download=True, transform=cifar10_transforms)
test_dataset = datasets.CIFAR10(data_directory, train=False, download=True, transform=cifar10_transforms)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.05,
    momentum=0.9,
    weight_decay=5e-4,
)

steps_per_epoch = 45000 // 256

trainer = Trainer(
    model=model,
    train_dataloader=train_dataloader,
    device="tpu",
    eval_dataloader=test_dataloader,
    optimizers=optimizer,
    max_duration='2ep',
    eval_interval=1,
    train_subset_num_batches=1,
)

trainer.fit()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/florescl/composer.git (to revision laura/tpu-perf) to /tmp/pip-req-build-lecamq7r
  Running command git clone -q https://github.com/florescl/composer.git /tmp/pip-req-build-lecamq7r
  Running command git checkout -b laura/tpu-perf --track origin/laura/tpu-perf
  Switched to a new branch 'laura/tpu-perf'
  Branch 'laura/tpu-perf' set up to track remote branch 'laura/tpu-perf' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 776.3 MB 10 kB/s 
  Created wheel for mosaicml: filename=mosaicml-0.8.1-py3-none-any.whl size=659540 sha256=37fce939941e74f757d4688069497385a6a6d592a509c630312c58faeafc4971
  Stored in directory: /tmp/pip-ephem-wheel-cache-dacpfxq0/wheels/e5/d4/68/ff8bdb31ca86458edc8c5a7db4ed317613b6232ac2621a371f
Successfully b

/usr/local/lib/python3.7/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


SyntaxError: ignored